In [9]:
import json
import os
import pandas as pd
from pycocotools.coco import COCO

In [128]:
# submission 양식에 맞게 output 후처리
def json_to_submission(json_file_path, submission_path, output_path, output_file_name, score_thr):

    with open(json_file_path, 'r') as f:
        data = json.load(f)

    coco = COCO(submission_path)
    img_ids = coco.getImgIds()

    prediction_strings = ['' for _ in range(len(img_ids))]
    file_names = [f'test/{str(img_id).zfill(4)}.jpg' for img_id in range(len(img_ids))]

    for info in data:

        img_id = info['image_id']
        xmin, ymin, w, h = info['bbox']
        xmax, ymax = xmin+w, ymin+h
        score = info['score']
        cate_id = info['category_id']

        if score < score_thr:
            continue

        prediction_list = map(str, [cate_id, score, xmin, ymin, xmax, ymax])
        prediction_string = ' '.join(prediction_list) + ' '

        prediction_strings[img_id] += prediction_string


    submission = pd.DataFrame()
    submission['PredictionString'] = prediction_strings
    submission['image_id'] = file_names

    submission.to_csv(os.path.join(output_path, output_file_name), index=None)
    print(submission.head())
    print(submission.tail())

In [133]:
json_file_path = './mmdetection_v3/work_dirs/h_dino_recycle/result/w_tta.bbox.json'
submission_path = './dataset/test.json'
output_path = './submission'
output_file_name = 'h_dino_result_tta_0.1.csv'
score_thr = 0.1

json_to_submission(json_file_path, submission_path, output_path, output_file_name, score_thr)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
                                    PredictionString       image_id
0  7 0.8832297325134277 603.5814208984375 519.343...  test/0000.jpg
1  5 0.6596419811248779 129.28302001953125 0.0 49...  test/0001.jpg
2  1 0.6677157282829285 77.11616516113281 273.067...  test/0002.jpg
3  9 0.8930801153182983 141.43505859375 263.62536...  test/0003.jpg
4  1 0.7861066460609436 189.751220703125 251.5643...  test/0004.jpg
                                       PredictionString       image_id
4866  0 0.6641607284545898 293.0938720703125 346.492...  test/4866.jpg
4867  3 0.7849198579788208 414.6515808105469 309.764...  test/4867.jpg
4868  7 0.8757544755935669 118.235595703125 474.3724...  test/4868.jpg
4869  0 0.6749555468559265 7.203704357147217 240.889...  test/4869.jpg
4870  4 0.8339611291885376 373.6348876953125 201.579...  test/4870.jpg
